# Unit 4: Add Custom Panels to RVIZ and Extras

Robotics is tough. Sometimes it's too Spartan, with only command executed programs. It always helps to have a simple GUI to execute the most used programs and give some fast interactivity.<br>
This is what you are going to learn here:

## Add a yes or no Interactive panel

This panel will return if you click yes or no on the panel. This is very useful for confirmation of actions, such as: Can I proceed with making the grasp?

The first step is to add the panel to the RVIZ. For that, you only have to go to <b>Panels</b> and <b>AddPanel</b>.

<table style="width:100%">
  <tr>
    <th>
    <figure>
      <img id="fig-A.1" src="img/rvizmarkers_unit4_addpanel.png" width="150"/>       
    </figure>

    </th>
    <th>
        <figure>
      <img id="fig-A.2" src="img/rvizmarkers_unit4_addpanelyes.png" width="300"/>
    </figure>
    </th>

  </tr>
</table>

Then, you have to create a program that calls a certain service and waits for the response. Here you have an example:

<p style="background:green;color:white;">panels_demo.py</p>

In [ ]:
#!/usr/bin/env python

import sys
import rospy
from jsk_gui_msgs.srv import YesNo, YesNoRequest, YesNoResponse


class RvizPanels(object):
    def __init__(self):
        self.yes_or_no_service_name = '/rviz/yes_no_button'
        rospy.wait_for_service(self.yes_or_no_service_name)
        rospy.loginfo(self.yes_or_no_service_name+"...Ready")
        
    def ask_yes_or_no_rviz(self):
    
        try:
            ask_yes_or_no_proxy = rospy.ServiceProxy(self.yes_or_no_service_name, YesNo)
            yes_no_req = YesNoRequest()
            responce = ask_yes_or_no_proxy(yes_no_req)
            return responce.yes
        except rospy.ServiceException, e:
            print "Service call failed: %s"%e


        

if __name__ == "__main__":
    rospy.init_node('demorvizpanels_node', anonymous=True)
    rvizpanels_object = RvizPanels()
    rospy.loginfo("Waiting for Answer in RVIZ panel")
    yes_answer = rvizpanels_object.ask_yes_or_no_rviz()
    rospy.loginfo("Answer Yes="+str(yes_answer))

<p style="background:green;color:white;">END panels_demo.py</p>

The service is always called <b>/rviz/yes_no_button</b>. It returns a service message of the type <b>YesNoResponse</b>. Here you have its structure:

As you can see, the request is Empty, and the response is a boolean; true if YES was clicked.

<p style="background:#EE9023;color:white;">Exercise U4-1</p>

Create a script that asks yes or no. If clicked, YES moves HaroRobot up and down. If not, then it moves it in circles.

<p style="background:#EE9023;color:white;">END Exercise U4-1</p>

## Add a custom GUI

So, yes or no interactivity is ok, but it quickly gets smaller for robotics applications. For that, you have the full panel experience.

### Step 0

You have to add the panel <b>RobotCommandInterfaceAction</b>:

<table style="width:100%">
  <tr>
    <th>
    <figure>
      <img id="fig-A.1" src="img/rvizmarkers_unit4_addpanel.png" width="150"/>       
    </figure>

    </th>
    <th>
        <figure>
      <img id="fig-A.2" src="img/rviz_marker_unit4_addcompletepanel.png" width="300"/>
    </figure>
    </th>

  </tr>
</table>

### Step 1

This starts with a config file like this one, <b>haro_robot_command.yaml</b>:

In [ ]:
robot_command_buttons:
  - name: "Execute Program"
    icon: "package://haro_description/img/myimage.jpg"
    type: euscommand
    command: "move_in_circles"
  - name: "Send EmptySrv message"
    icon: "package://haro_description/img/myimage2.png"
    type: emptysrv
    srv: "/move_up_and_down"

You can use these interactive panels in two ways:<br>
<ul>
<li>Using EmptySrv messages: This will send an emptysrv.srv message to the service stated in the <b>srv</b> tag. It's useful if only the activation of something is needed.</li>
<li>Using euscommand type messages: This is used when you want to send data to the executing system. You have to use a service that HAS to be named <b>euscommand</b>, and of the EusCommand.srv type. It then allows you to send any information in string format through the field <b>command</b>.</li>
</ul>

In both, you can state which icon is to be placed, retrieving it from a package. You can also state the name that will appear under the button.

### Step 2

Create a server that can process the EusCommand messages. It has to be like this one:

In [ ]:
service_server_name = "/eus_command"
s = rospy.Service(service_server_name, EusCommand, handle_command)

handle_command(req):
    """
    [jsk_rviz_plugins/EusCommand.srv]
    string command                                                                                                                 
    ---  
    """
    rospy.loginfo("Command Requested==>"+str(req.command))
    if req.command == "move_in_circles":
        self.haro_object.move_for_x_time(num=20)
    return EusCommandResponse()


In this case with this service, if the string given is <b>move_in_circles</b>, it will move Haro in circles.

### Step 3

Create a launch file that launches the server and loads the <b>haro_robot_command.yaml<b>.

This will launch RVIZ with a session of RVIZ that should have the panel activated. It will then launch the server that will listen to the buttons selected.

<p style="background:green;color:white;">robot_command_server.py</p>

In [ ]:
#!/usr/bin/env python

import rospy
from jsk_rviz_plugins.srv import EusCommand, EusCommandRequest, EusCommandResponse 
from std_srvs.srv import Empty, EmptyRequest, EmptyResponse
from move_haro_in_circles import HaroMove

class RobotCommandServer(object):
    def __init__(self):
        # To make the robot command RVIZ panel work it has to be this name
        service_server_name = "/eus_command"
        s = rospy.Service(service_server_name, EusCommand, self.handle_command)
        rospy.loginfo("Service =>"+str(service_server_name)+",Ready to recieve commands...")
        
        updown_service_server_name = "/move_up_and_down"
        s = rospy.Service(updown_service_server_name, Empty, self.handle_updown_command)
        rospy.loginfo("Service =>"+str(updown_service_server_name)+",Ready to recieve commands...")
        
        self.haro_object = HaroMove()
        

    def handle_command(self,req):
        """
        [jsk_rviz_plugins/EusCommand.srv]
        string command                                                                                                                 
        ---  
        """
        rospy.loginfo("Command Requested==>"+str(req.command))
        if req.command == "move_in_circles":
            self.haro_object.move_for_x_time(num=20)
        return EusCommandResponse()
        
    def handle_updown_command(self,req):
        """
        [std_srvs/Empty.srv]
        ---  
        """
        rospy.loginfo("Move Up And Down")
        self.haro_object.move_for_x_time_updown(num=100)
        return EmptyResponse()
    
    def start_server_spin(self):
        rospy.spin()



if __name__ == "__main__":
    rospy.init_node('robot_command_server')
    robotserver_object = RobotCommandServer()
    robotserver_object.start_server_spin()

<p style="background:green;color:white;">END robot_command_server.py</p>

Here you have an example of the python module that makes Haro move in circles, and up and down.

<p style="background:green;color:white;">move_haro_in_circles.py</p>

In [ ]:
#!/usr/bin/env python
import rospy
from geometry_msgs.msg import Twist

class HaroMove():
    
    def __init__(self):
        self.haro_vel = rospy.Publisher('/haro/cmd_vel', Twist, queue_size=1)
        self.ctrl_c = False
        rospy.on_shutdown(self.shutdownhook)
        self.rate = rospy.Rate(10) # 10hz
    
    def shutdownhook(self):
            # works better than the rospy.is_shut_down()
            print "shutdown time! Stop the robot"
            cmd = Twist()
            cmd.linear.x = 0.0
            cmd.angular.z = 0.0
            self.haro_vel.publish(cmd)
            self.ctrl_c = True

    def start(self):
        cmd = Twist()
        cmd.linear.x = 1.0
        cmd.angular.z = 1.0
        while not self.ctrl_c:
            self.haro_vel.publish(cmd)
            self.rate.sleep()
    
    def move_for_x_time(self, num=20):
        cmd = Twist()
        cmd.linear.x = 1.0
        cmd.angular.z = 1.0
        i = 0
        while i <= num:
            rospy.loginfo(i)
            self.haro_vel.publish(cmd)
            self.rate.sleep()
            i += 1
        self.shutdownhook()
            
    def move_for_x_time_updown(self, num=20):
        cmd = Twist()
        i = 0
        while i <= num:
            if i > num/2.0:
                cmd.linear.z = -0.5
                rospy.loginfo("Move Down")
            else:
                cmd.linear.z = 0.5
                rospy.loginfo("Move Up")
            self.haro_vel.publish(cmd)
            self.rate.sleep()
            i += 1
        self.shutdownhook()

if __name__ == '__main__':
    rospy.init_node('move_haro_in_circles_node', anonymous=True)
    haro_object = HaroMove()
    try:
        haro_object.move_for_x_time(num=20)
    except rospy.ROSInterruptException:
        pass

<p style="background:green;color:white;">END move_haro_in_circles.py</p>

You should get something similar to this:

<img src="img/rvizmarkers_unit4_custompanel.gif"/>

<p style="background:#EE9023;color:white;">Exercise U4-2</p>

Create your own Custom Panel that has the following features:<br>
<ul>
<li>Has Three Buttons</li>
<li>You can make Haro turn itself on</li>
<li>You can make the StandingPerson move in circles</li>
<li>You can change the pressing of one of the buttons of the Pictogram on top of Haro</li>
</ul>

<p style="background:#EE9023;color:white;">END Exercise U4-2</p>

## Draw Trajectories and TwistStamped in RVIZ

This is super simple. Just add the following elements and you will have represented the TF trajectories and the TwistStamped sent to move Haro.<br>
You have to add the <b>TFTrajectory</b> and select the frame you want to draw the trajectory.<br>
You have to add the <b>TwistStamped</b> and select the topic of the type TwistStamped. In this case, it is <b>rostopic echo /haro/twist_stamped_publisher</b>.

<img src="img/rvizmarkers_unit4_addtraj.png"/>

You should then see something similar to this when you move Haro through the command:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p><br>
roslaunch haro_description move_haro_keyboard.launch
</th>
</tr>
</table>

If you press the keys continuously, the TwistStamped arrows will stay there until you stop pressing.

<img src="img/rvizmarkers_unit4_stamptraj.gif"/>

## Record RVIZ Sessions

Add the <b>VideoCapture</b> element to be able to record videos by just checking the <b>start_capture box</b>.<br>
To stop the video, just uncheck the <b>start_capture box</b>. It's that simple.

<img src="img/rvizmarkers_unit4_record.png"/>

<p style="background:blue;color:white;">You have to state the <b>filename,</b> which is the path to record to. Be careful because these files grow very quickly in size.</p>

## Congratulations! You have finished the RVIZ Markers course!

What's next? Continue your ROS learning and Robotics projects with us.